In [4]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [6]:
subject = 'Pernambuco - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [7]:
data = pd.read_csv('2003_01_model_input_PE.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Pernambuco - Valor Cimento Portland,Pernambuco - IDH,Pernambuco - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pernambuco - Desemprego,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - Consumo de Cimento (t)
0,2003,3.627100,0.688685,425.758313,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,...,27.054426,21.478883,72.419858,2.181139,8.294029,8.237381e+07,5.574893e+06,8.646979,7.571061e+07,1013.872000
1,2004,3.613756,0.692578,463.749993,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,...,26.761671,21.348614,73.212023,2.102787,8.221752,8.366033e+07,5.640541e+06,8.694752,7.623567e+07,951.037000
2,2005,3.503074,0.696127,471.505719,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,...,25.833659,20.901231,75.078892,2.650050,8.199101,8.484033e+07,5.649536e+06,8.733906,7.666319e+07,979.656000
3,2006,3.449433,0.699339,514.663879,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,...,25.630302,20.815358,75.254342,2.832919,8.107273,8.594487e+07,5.677153e+06,8.770881,7.705796e+07,1193.338000
4,2007,3.480239,0.702226,623.964785,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,...,25.669231,20.900938,74.670522,2.505889,8.081524,8.696379e+07,5.727693e+06,8.804784,7.741156e+07,1320.587000
5,2008,3.575368,0.704795,722.146507,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,...,25.871219,21.250711,75.419489,2.117760,8.049769,8.785657e+07,5.769035e+06,8.834432,7.768216e+07,1717.153000
6,2009,3.706614,0.707057,803.348817,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,...,25.874600,21.577558,77.766757,1.951382,8.078293,8.860216e+07,5.810149e+06,8.858245,7.790384e+07,1878.938000
7,2010,3.890411,0.709020,950.406487,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,...,25.750844,21.892207,80.130602,1.854110,8.042187,9.096526e+07,5.826469e+06,8.909890,7.856512e+07,2406.931000
8,2011,4.085239,0.710693,1079.328424,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,...,25.736537,21.841193,80.177776,1.815157,8.028882,9.301827e+07,5.792701e+06,8.937706,7.897343e+07,2496.113000
9,2012,4.276735,0.711600,1132.134532,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,...,25.711449,21.251417,77.642932,1.728390,12.192814,9.484266e+07,5.717659e+06,8.949260,7.926901e+07,2728.230000


In [8]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Pernambuco - Valor Cimento Portland,Pernambuco - IDH,Pernambuco - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pernambuco - Desemprego,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado
0,3.627100,0.688685,425.758313,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,...,1823.059800,27.054426,21.478883,72.419858,2.181139,8.294029,8.237381e+07,5.574893e+06,8.646979,7.571061e+07
1,3.613756,0.692578,463.749993,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,...,1802.115999,26.761671,21.348614,73.212023,2.102787,8.221752,8.366033e+07,5.640541e+06,8.694752,7.623567e+07
2,3.503074,0.696127,471.505719,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,...,1776.928825,25.833659,20.901231,75.078892,2.650050,8.199101,8.484033e+07,5.649536e+06,8.733906,7.666319e+07
3,3.449433,0.699339,514.663879,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,...,1779.983604,25.630302,20.815358,75.254342,2.832919,8.107273,8.594487e+07,5.677153e+06,8.770881,7.705796e+07
4,3.480239,0.702226,623.964785,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,...,1696.205011,25.669231,20.900938,74.670522,2.505889,8.081524,8.696379e+07,5.727693e+06,8.804784,7.741156e+07
5,3.575368,0.704795,722.146507,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,...,1564.493495,25.871219,21.250711,75.419489,2.117760,8.049769,8.785657e+07,5.769035e+06,8.834432,7.768216e+07
6,3.706614,0.707057,803.348817,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,...,1487.197796,25.874600,21.577558,77.766757,1.951382,8.078293,8.860216e+07,5.810149e+06,8.858245,7.790384e+07
7,3.890411,0.709020,950.406487,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,...,1437.348866,25.750844,21.892207,80.130602,1.854110,8.042187,9.096526e+07,5.826469e+06,8.909890,7.856512e+07
8,4.085239,0.710693,1079.328424,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,...,1436.270203,25.736537,21.841193,80.177776,1.815157,8.028882,9.301827e+07,5.792701e+06,8.937706,7.897343e+07
9,4.276735,0.711600,1132.134532,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,...,1436.469748,25.711449,21.251417,77.642932,1.728390,12.192814,9.484266e+07,5.717659e+06,8.949260,7.926901e+07


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      951.037000
1      979.656000
2     1193.338000
3     1320.587000
4     1717.153000
5     1878.938000
6     2406.931000
7     2496.113000
8     2728.230000
9     2982.021000
10    2983.632857
11    2632.837886
12    2308.455257
13    1813.941000
14    1645.004000
15    1690.846000
16    2018.834000
17    2022.411000
18    1940.927000
19            NaN
Name: Pernambuco - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Pernambuco - Valor Cimento Portland,Pernambuco - IDH,Pernambuco - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pernambuco - Desemprego,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado
0,-0.909391,-1.964470,-1.047122,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,...,1.294775,2.994393,0.659857,-1.936308,0.518418,-0.867275,-1.686779,-0.132100,-2.296300,-2.161094
1,-0.941707,-1.590070,-0.902028,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,...,1.140837,2.206082,0.252353,-1.541636,0.311267,-0.891581,-1.460188,0.258574,-1.741827,-1.695260
2,-1.209744,-1.248837,-0.872408,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,...,0.955709,-0.292800,-1.147138,-0.611527,1.758156,-0.899198,-1.252358,0.312107,-1.287396,-1.315955
3,-1.339644,-0.939881,-0.707581,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,...,0.978162,-0.840386,-1.415763,-0.524115,2.241636,-0.930080,-1.057817,0.476457,-0.858248,-0.965711
4,-1.265042,-0.662309,-0.290148,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,...,0.362385,-0.735560,-1.148055,-0.814984,1.377013,-0.938739,-0.878357,0.777220,-0.464759,-0.651997
5,-1.034671,-0.415233,0.084820,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,...,-0.605703,-0.191663,-0.053903,-0.441835,0.350852,-0.949418,-0.721114,1.023251,-0.120648,-0.411915
6,-0.716834,-0.197759,0.394942,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,...,-1.173830,-0.182557,0.968527,0.727618,-0.089028,-0.939826,-0.589796,1.267927,0.155728,-0.215239
7,-0.271738,-0.008998,0.956573,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,...,-1.540223,-0.515799,1.952803,1.905330,-0.346203,-0.951968,-0.173588,1.365048,0.755145,0.371461
8,0.200072,0.151941,1.448942,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,...,-1.548151,-0.554324,1.793221,1.928833,-0.449188,-0.956442,0.188003,1.164091,1.077983,0.733719
9,0.663815,0.239129,1.650615,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,...,-1.546685,-0.621880,-0.051695,0.665926,-0.678589,0.443866,0.509330,0.717508,1.212088,0.995958


In [11]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      951.037000
1      979.656000
2     1193.338000
3     1320.587000
4     1717.153000
5     1878.938000
6     2406.931000
7     2496.113000
8     2728.230000
9     2982.021000
10    2983.632857
11    2632.837886
12    2308.455257
13    1813.941000
14    1645.004000
15    1690.846000
16    2018.834000
Name: Pernambuco - Consumo de Cimento (t), dtype: float64

In [12]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Pernambuco - Valor Cimento Portland,Pernambuco - IDH,Pernambuco - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pernambuco - Desemprego,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado
17,2.140266,0.998695,-1.205634,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,...,-0.400092,-0.174083,1.094154,1.140813,0.074891,1.070627,0.666787,-0.999485,-0.451599,-0.024935


In [13]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    2022.411
Name: Pernambuco - Consumo de Cimento (t), dtype: float64

In [14]:
train_input.iloc[-(len(train_input)//5):]

,Pernambuco - Valor Cimento Portland,Pernambuco - IDH,Pernambuco - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pernambuco - Desemprego,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado
14,0.955786,1.317253,-0.999691,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,...,0.894667,-0.237014,0.089836,0.409915,-1.073923,1.246751,1.169858,-1.492793,0.419587,0.747715
15,1.424563,1.324346,-1.196005,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,...,0.345540,0.084575,0.339945,0.334279,-0.227765,1.162017,1.086335,-1.452034,0.155140,0.518015
16,1.800978,1.218437,-1.203936,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,...,-0.252078,0.224133,0.633408,0.458452,0.449563,1.147174,0.944355,-1.289950,-0.105390,0.300429


In [15]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [18]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [19]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3938726147, 2592122079, 2766471204, 1217315521, 918407772, 2760494107, 457354134, 3710021748, 3512789864, 83527662, 3158802076, 2774870776, 2647595382, 3182788114, 1611113365, 3047061047, 1652568326, 3403249721, 1666471486, 211530464, 4106070669, 2908942728, 220744578, 1285779085, 3988234965, 36863100, 1142328099, 311598506, 2196465071, 2731355758, 1821340295, 3870927096, 4153598039, 2317502821, 818619092, 3738506752, 2146876842, 3915261740, 2868403909, 2579081083, 2474214414, 428626448, 4190406878, 3514182097, 3408992882, 2572535369, 3797171191, 1211153289, 3721368781, 1573113384, 3601172170, 1075377025, 1544653545, 3972311645, 2042094151, 2132181097, 1961977260, 29246054, 2056546580, 15969623, 686633787, 1112049513, 1293538893, 117898738, 2152539744, 3985497071, 161495537, 1177153254, 2246886520, 72987549, 1007402701, 3175872696, 3289428241, 1292064535, 170843929, 3206031131, 3526435527, 1115793252, 1771009898, 3382192360, 2859558413, 1711329217, 3704891572, 1899755798, 3953009719, 

val_loss: 7830.35986328125


Step: 88 ___________________________________________
val_loss: 29839.1796875


Step: 89 ___________________________________________
val_loss: 27755.537109375


Step: 90 ___________________________________________
val_loss: 55639.35546875


Step: 91 ___________________________________________
val_loss: 52601.34765625


Step: 92 ___________________________________________
val_loss: 30943.306640625


Step: 93 ___________________________________________
val_loss: 90225.15625


Step: 94 ___________________________________________
val_loss: 39110.00390625


Step: 95 ___________________________________________
val_loss: 43078.33984375


Step: 96 ___________________________________________
val_loss: 27725.4375


Step: 97 ___________________________________________
val_loss: 27755.84375


Step: 98 ___________________________________________
val_loss: 43030.953125


Step: 99 ___________________________________________
val_loss: 27975.166015625


final_seed: 138645305

In [20]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 464ms/step - loss: 4350825.5000 - val_loss: 3152746.0000
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 4224995.5000 - val_loss: 836607.2500
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 3531122.0000 - val_loss: 562797.0625
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 3178425.7500 - val_loss: 526673.7500
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 4423319.5000 - val_loss: 1530924.6250
Epoch 6/10000
1/1 [==============================] - 0s 15ms/step - loss: 3743896.5000 - val_loss: 1667479.5000
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 3067368.7500 - val_loss: 1895180.3750
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 2448733.2500 - val_loss: 1020318.6875
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 2359211.0000 - val_loss: 1157597

1/1 [==============================] - 0s 14ms/step - loss: 873065.4375 - val_loss: 226244.6719
Epoch 75/10000
1/1 [==============================] - 0s 14ms/step - loss: 722892.3750 - val_loss: 89357.6953
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 638895.8750 - val_loss: 90331.6016
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 707932.0000 - val_loss: 122676.3516
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 799722.6250 - val_loss: 156901.2031
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 608583.4375 - val_loss: 99305.2500
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 757333.9375 - val_loss: 158037.7969
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 777661.8125 - val_loss: 151213.7031
Epoch 82/10000
1/1 [==============================] - 0s 14ms/step - loss: 579830.5000 - val_loss: 99874.9609
Epoch 83/10000
1/1 [

1/1 [==============================] - 0s 14ms/step - loss: 301690.3438 - val_loss: 49199.5586
Epoch 149/10000
1/1 [==============================] - 0s 15ms/step - loss: 326821.8125 - val_loss: 54880.8008
Epoch 150/10000
1/1 [==============================] - 0s 14ms/step - loss: 445383.7188 - val_loss: 57483.9258
Epoch 151/10000
1/1 [==============================] - 0s 14ms/step - loss: 323209.5625 - val_loss: 48793.5156
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 344994.8750 - val_loss: 62021.8164
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 336390.1562 - val_loss: 43158.8828
Epoch 154/10000
1/1 [==============================] - 0s 14ms/step - loss: 318211.5312 - val_loss: 54092.5117
Epoch 155/10000
1/1 [==============================] - 0s 14ms/step - loss: 312400.7188 - val_loss: 42722.8008
Epoch 156/10000
1/1 [==============================] - 0s 14ms/step - loss: 295847.9375 - val_loss: 52364.4883
Epoch 157/10000
1

Epoch 222/10000
1/1 [==============================] - 0s 14ms/step - loss: 173631.0156 - val_loss: 31726.9922
Epoch 223/10000
1/1 [==============================] - 0s 14ms/step - loss: 151829.1875 - val_loss: 31632.5332
Epoch 224/10000
1/1 [==============================] - 0s 14ms/step - loss: 168132.5625 - val_loss: 31586.6973
Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 192589.6875 - val_loss: 32298.6934
Epoch 226/10000
1/1 [==============================] - 0s 15ms/step - loss: 176742.6406 - val_loss: 30503.1465
Epoch 227/10000
1/1 [==============================] - 0s 14ms/step - loss: 160824.6406 - val_loss: 29656.7402
Epoch 228/10000
1/1 [==============================] - 0s 14ms/step - loss: 175303.6250 - val_loss: 28321.9941
Epoch 229/10000
1/1 [==============================] - 0s 15ms/step - loss: 187398.3750 - val_loss: 30236.2090
Epoch 230/10000
1/1 [==============================] - 0s 15ms/step - loss: 182915.7500 - val_loss: 31327.0723
E

Epoch 296/10000
1/1 [==============================] - 0s 14ms/step - loss: 113324.0625 - val_loss: 62775.8438
Epoch 297/10000
1/1 [==============================] - 0s 14ms/step - loss: 122154.9141 - val_loss: 41826.0820
Epoch 298/10000
1/1 [==============================] - 0s 14ms/step - loss: 132966.3438 - val_loss: 50785.8906
Epoch 299/10000
1/1 [==============================] - 0s 14ms/step - loss: 194967.0938 - val_loss: 35755.3594
Epoch 300/10000
1/1 [==============================] - 0s 14ms/step - loss: 136821.0312 - val_loss: 56963.0273
Epoch 301/10000
1/1 [==============================] - 0s 14ms/step - loss: 123669.0703 - val_loss: 62485.2852
Epoch 302/10000
1/1 [==============================] - 0s 14ms/step - loss: 126868.9844 - val_loss: 63921.4258
Epoch 303/10000
1/1 [==============================] - 0s 14ms/step - loss: 171982.0625 - val_loss: 42710.0469
Epoch 304/10000
1/1 [==============================] - 0s 14ms/step - loss: 119920.8125 - val_loss: 61949.5312
E

...

1/1 [==============================] - 0s 14ms/step - loss: 80371.2891 - val_loss: 158552.1250
Epoch 888/10000
1/1 [==============================] - 0s 14ms/step - loss: 67862.5156 - val_loss: 49353.0977
Epoch 889/10000
1/1 [==============================] - 0s 14ms/step - loss: 119140.0859 - val_loss: 133791.2500
Epoch 890/10000
1/1 [==============================] - 0s 14ms/step - loss: 78733.1484 - val_loss: 122233.5938
Epoch 891/10000
1/1 [==============================] - 0s 14ms/step - loss: 90112.0703 - val_loss: 125349.3203
Epoch 892/10000
1/1 [==============================] - 0s 15ms/step - loss: 116426.6953 - val_loss: 114709.0234
Epoch 893/10000
1/1 [==============================] - 0s 14ms/step - loss: 93496.3047 - val_loss: 134703.0156
Epoch 894/10000
1/1 [==============================] - 0s 14ms/step - loss: 91886.9062 - val_loss: 93286.8672
Epoch 895/10000
1/1 [==============================] - 0s 14ms/step - loss: 76657.1562 - val_loss: 146954.5312
Epoch 896/10000
1

Epoch 961/10000
1/1 [==============================] - 0s 14ms/step - loss: 89868.7344 - val_loss: 138181.2656
Epoch 962/10000
1/1 [==============================] - 0s 14ms/step - loss: 101462.2266 - val_loss: 90536.1797
Epoch 963/10000
1/1 [==============================] - 0s 14ms/step - loss: 91430.1641 - val_loss: 104304.6797
Epoch 964/10000
1/1 [==============================] - 0s 14ms/step - loss: 136219.1094 - val_loss: 103371.7500
Epoch 965/10000
1/1 [==============================] - 0s 14ms/step - loss: 118069.0234 - val_loss: 76918.0391
Epoch 966/10000
1/1 [==============================] - 0s 14ms/step - loss: 95856.0000 - val_loss: 122831.1172
Epoch 967/10000
1/1 [==============================] - 0s 14ms/step - loss: 87326.6484 - val_loss: 120789.1797
Epoch 968/10000
1/1 [==============================] - 0s 14ms/step - loss: 83704.6875 - val_loss: 138617.4375
Epoch 969/10000
1/1 [==============================] - 0s 14ms/step - loss: 103355.4688 - val_loss: 100469.1797

Epoch 1035/10000
1/1 [==============================] - 0s 14ms/step - loss: 66533.1875 - val_loss: 60523.6367
Epoch 1036/10000
1/1 [==============================] - 0s 14ms/step - loss: 83301.5312 - val_loss: 40595.8594
Epoch 1037/10000
1/1 [==============================] - 0s 14ms/step - loss: 70121.3828 - val_loss: 68725.0938
Epoch 1038/10000
1/1 [==============================] - 0s 14ms/step - loss: 101816.9062 - val_loss: 38842.9375
Epoch 1039/10000
1/1 [==============================] - 0s 14ms/step - loss: 93276.7500 - val_loss: 67248.5703
Epoch 1040/10000
1/1 [==============================] - 0s 14ms/step - loss: 64532.5664 - val_loss: 66295.8516
Epoch 1041/10000
1/1 [==============================] - 0s 14ms/step - loss: 103977.7344 - val_loss: 41665.3477
Epoch 1042/10000
1/1 [==============================] - 0s 14ms/step - loss: 69172.4531 - val_loss: 34908.4336
Epoch 1043/10000
1/1 [==============================] - 0s 14ms/step - loss: 55946.1523 - val_loss: 49123.8633

Epoch 1109/10000
1/1 [==============================] - 0s 14ms/step - loss: 167170.3125 - val_loss: 68608.6016
Epoch 1110/10000
1/1 [==============================] - 0s 14ms/step - loss: 90599.4453 - val_loss: 68442.1875
Epoch 1111/10000
1/1 [==============================] - 0s 15ms/step - loss: 71887.4922 - val_loss: 75840.4062
Epoch 1112/10000
1/1 [==============================] - 0s 14ms/step - loss: 76737.5938 - val_loss: 87287.1641
Epoch 1113/10000
1/1 [==============================] - 0s 14ms/step - loss: 130134.2969 - val_loss: 66764.0469
Epoch 1114/10000
1/1 [==============================] - 0s 14ms/step - loss: 96771.7031 - val_loss: 95012.8750
Epoch 1115/10000
1/1 [==============================] - 0s 14ms/step - loss: 105396.8594 - val_loss: 77271.2734
Epoch 1116/10000
1/1 [==============================] - 0s 14ms/step - loss: 75299.7031 - val_loss: 91508.0938
Epoch 1117/10000
1/1 [==============================] - 0s 14ms/step - loss: 106947.8516 - val_loss: 50677.62

1/1 [==============================] - 0s 14ms/step - loss: 121005.8516 - val_loss: 111088.0078
Epoch 1183/10000
1/1 [==============================] - 0s 14ms/step - loss: 98270.7734 - val_loss: 97871.5859
Epoch 1184/10000
1/1 [==============================] - 0s 14ms/step - loss: 94047.0234 - val_loss: 136955.5156
Epoch 1185/10000
1/1 [==============================] - 0s 14ms/step - loss: 104397.9141 - val_loss: 80864.5078
Epoch 1186/10000
1/1 [==============================] - 0s 14ms/step - loss: 77786.3906 - val_loss: 110705.5234
Epoch 1187/10000
1/1 [==============================] - 0s 14ms/step - loss: 87740.9219 - val_loss: 130923.6875
Epoch 1188/10000
1/1 [==============================] - 0s 14ms/step - loss: 106904.9141 - val_loss: 107518.9922
Epoch 1189/10000
1/1 [==============================] - 0s 14ms/step - loss: 77815.8203 - val_loss: 111290.5859
Epoch 1190/10000
1/1 [==============================] - 0s 14ms/step - loss: 71497.9375 - val_loss: 108047.6016
Epoch 11

In [21]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 49ms/step
[2022.411](test_target) - [[2128.104]](prediction) = 105.69300390624994


In [22]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [23]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Pernambuco - Valor Cimento Portland,Pernambuco - IDH,Pernambuco - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pernambuco - Desemprego,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado
0,0.159485,-1.689052,-1.113145,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,...,1.063370,1.979997,0.585709,-1.337959,-0.286379,1.850585,-1.543090,-1.619622,-1.644505,-1.656003
1,0.059807,-1.106763,-0.897581,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,...,0.916988,1.399577,0.219228,-0.993192,-0.526708,1.013504,-1.050126,-0.829344,-1.061495,-1.053508
2,-0.766956,-0.576057,-0.853576,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,...,0.740948,-0.440307,-1.039378,-0.180690,1.151910,0.751177,-0.597974,-0.721056,-0.583677,-0.562927
3,-1.167633,-0.095549,-0.608698,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,...,0.762298,-0.843485,-1.280961,-0.104331,1.712822,-0.312326,-0.174736,-0.388598,-0.132443,-0.109933
4,-0.937523,0.336147,0.011471,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,...,0.176747,-0.766304,-1.040202,-0.358422,0.709724,-0.610536,0.215691,0.219803,0.281298,0.295815
5,-0.226941,0.720415,0.568550,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,...,-0.743821,-0.365842,-0.056198,-0.032455,-0.480782,-0.978316,0.557787,0.717490,0.643118,0.606329
6,0.753429,1.058643,1.029289,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,...,-1.284061,-0.359137,0.863306,0.989126,-0.991112,-0.647965,0.843479,1.212433,0.933718,0.860704
7,2.126333,1.352216,1.863689,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,...,-1.632469,-0.604498,1.748496,2.017923,-1.289475,-1.066123,1.748970,1.408895,1.563986,1.619522


train_target:


,Pernambuco - Consumo de Cimento (t)
0,951.037
1,979.656
2,1193.338
3,1320.587
4,1717.153
5,1878.938
6,2406.931
7,2496.113


test_input:


...

,Pernambuco - Valor Cimento Portland,Pernambuco - IDH,Pernambuco - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pernambuco - Desemprego,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado
17,2.140266,0.998695,-1.205634,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,...,-0.400092,-0.174083,1.094154,1.140813,0.074891,1.070627,0.666787,-0.999485,-0.451599,-0.024935


test_target:


,Pernambuco - Consumo de Cimento (t)
17,2022.411


1/1 [==============================] - 0s 35ms/step
Error: 105.69300390624994




[2495.201171875,
 2665.3447265625,
 2817.943359375,
 2480.511474609375,
 2667.21728515625,
 2530.8671875,
 2089.6162109375,
 2182.32275390625,
 1869.88916015625,
 2128.10400390625]

In [24]:
display(targets)
display(predictions)

[2728.23,
 2982.021,
 2983.632857142857,
 2632.8378857142857,
 2308.455257142857,
 1813.941,
 1645.004,
 1690.846,
 2018.834,
 2022.411]

[2495.201171875,
 2665.3447265625,
 2817.943359375,
 2480.511474609375,
 2667.21728515625,
 2530.8671875,
 2089.6162109375,
 2182.32275390625,
 1869.88916015625,
 2128.10400390625]

In [25]:
mae = mean_absolute_error(predictions, targets)
mae

313.41360345424107

In [26]:
porcentage = mae/np.mean(targets)
porcentage

0.13730424904658564